In [65]:
#https://github.com/Nithyashree-2022/VGG-19-for-Rock-Paper-and-Scissors-classification/blob/main/VGG19%20for%20RPS-80%2020%20split-FINAL.ipynb
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [66]:
IMAGE_SIZE = [64, 64]

In [75]:
train_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set"
test_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set"
val_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set"

In [76]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(64,64))
        x_train.append(img_arr)

/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set/Hispa
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set/LeafBlast
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set/BrownSpot
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set/Healthy


In [92]:
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(64,64))
        x_test.append(img_arr)

/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set/Hispa
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set/LeafBlast
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set/BrownSpot
/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set/Healthy


In [93]:
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(64,64))
        x_val.append(img_arr)

In [94]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [95]:
train_x.shape,test_x.shape,val_x.shape


((608, 64, 64, 3), (100, 64, 64, 3), (100, 64, 64, 3))

In [96]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [97]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [98]:
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 608 images belonging to 4 classes.
Found 100 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [99]:
training_set.class_indices


{'BrownSpot': 0, 'Healthy': 1, 'Hispa': 2, 'LeafBlast': 3}

In [100]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

((608,), (100,), (100,))

In [101]:
# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [102]:
for layer in vgg.layers:
    layer.trainable = False

In [107]:
x = Flatten()(vgg.output)

prediction = Dense(4, activation='softmax')(x)

In [108]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [118]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [123]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=100,
  
  batch_size=32,shuffle=True)

Epoch 1/100
 2/19 [==>...........................] - ETA: 14s - loss: 0.4610 - accuracy: 0.8125

KeyboardInterrupt: 